In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%bash
# Logistics #2: install the transformers package, create a folder, download the dataset and a patch
pip -q install transformers

# remove the directory if necessary
# rm -rf "/content/gdrive/MyDrive/6864_hw4/"

mkdir "/content/drive/My Drive/NLP/"
cd "/content/drive/My Drive/NLP/"

mkdir: cannot create directory ‘/content/drive/My Drive/NLP/’: File exists


In [3]:
import numpy as np
import pandas as pd
import torch

if torch.cuda.is_available():
  device = torch.device('cuda')

  print('There are %d GPU(s) available.' % torch.cuda.device_count())

  print('We will use the GPU:', torch.cuda.get_device_name(0))

There are 1 GPU(s) available.
We will use the GPU: Tesla P100-PCIE-16GB


In [4]:
project_path = '/content/drive/My Drive/NLP/'
dataset = pd.read_csv(project_path + 'fake_and_real_news/combined.csv')
# dataset['text'] = dataset['text'].str.replace("^.*\(Reuters\) - ", "", regex=True)


In [5]:
import transformers

tokenizer = transformers.AutoTokenizer.from_pretrained('roberta-base')
tokenizer('hello world')

{'input_ids': [0, 42891, 232, 2], 'attention_mask': [1, 1, 1, 1]}

In [6]:
import torch.nn as nn

class FakeNewsClassifier(nn.Module):
  def __init__(self, lm, dropout=0.2):
    super(FakeNewsClassifier, self).__init__()
    self.lm = lm
    self.dropout = nn.Dropout(dropout)
    self.linear = nn.Linear(lm.config.hidden_size, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, input_ids, attention_mask=None):
    lm_output = self.lm(input_ids, attention_mask)
    pooled = lm_output.pooler_output
    pooled = self.dropout(pooled)
    lin_output = self.linear(pooled)
    pred = self.sigmoid(lin_output)
    return pred
  


In [7]:
roberta = transformers.AutoModel.from_pretrained('roberta-base')
model = FakeNewsClassifier(roberta)
model = model.cuda()

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
def tokenize_batch_farn(batch):
  x = list(batch['text'])
  y = np.array(batch['label'])
  
  x_encode = tokenizer.batch_encode_plus(
      x,
      max_length=512,
      truncation=True,
      padding='longest',
      return_attention_mask=True,
      return_tensors='pt'
  )
  input_ids = x_encode['input_ids'].cuda()
  input_mask = x_encode['attention_mask'].cuda()

  return input_ids, input_mask, y

In [9]:

training_size = int(.6*dataset.shape[0])
validation_size = int(.2*dataset.shape[0])
testing_size = int(.2*dataset.shape[0])

data_train = dataset[:training_size]
data_val = dataset[training_size:training_size+validation_size]
data_test = dataset[training_size+validation_size:]

print(len(data_train))
print(len(data_val))
print(len(data_test))


26938
8979
8981


In [10]:
def decode(pred):
  decoded = torch.where(pred >= 0.5, 1, 0)
  return decoded

def get_acc(preds, labels):
  decoded = decode(preds).to(dtype=torch.float32)
  acc = torch.where(decoded == labels, 1, 0)
  return acc.sum()

In [12]:

import matplotlib.pyplot as plt

num_epochs = 1
learning_rate = 3e-5
weight_decay = 1e-5
eps = 1e-6
batch_size = 16
warmup_rate = 0.05

# Initializing an AdamW optimizer
ext_optim = torch.optim.AdamW(model.parameters(), lr=learning_rate,
                              eps=eps, weight_decay=weight_decay)

loss_fn = nn.BCELoss()

model.train()

val_accuracies = []
for _ in range(num_epochs):
  for i in range(0, len(data_train), batch_size):
    batch = data_train[i : i + batch_size]
    input_ids, input_mask, labels = tokenize_batch_farn(batch)
    labels = torch.tensor(labels).unsqueeze(-1).to(dtype=torch.float32).cuda()

    model.zero_grad()
    predictions = model(input_ids, input_mask)
    predictions.to(torch.float32)

    loss = loss_fn(predictions, labels)
    loss.backward()

    ext_optim.step()

    if i % 1000 == 0:
    #   total_val_acc = 0
    #   val_batch_size = 1
    #   for i in range(0, len(data_val), val_batch_size):
    #     input_ids, input_mask, labels = tokenize_batch_farn(data_val[i : i+val_batch_size])
    #     labels = torch.tensor(labels).unsqueeze(-1).to(dtype=torch.float32).cuda()
    #     preds = model.forward(input_ids, input_mask)
    #     decoded = decode(preds).to(dtype=torch.float32)
    #     acc = get_acc(decoded, labels)
    #     total_val_acc += acc
    #   total_val_acc = int(total_val_acc) / len(data_val)
    #   val_accuracies.append(total_val_acc)

      print("Loss: {}".format(loss))
    #   print("Val acc: {}".format(total_val_acc))

# plt.plot(val_accuracies)
# plt.show()

torch.save(model.state_dict(), project_path+'roberta/model_embed_2.pt')




Loss: 0.03651557117700577
Loss: 0.0006971255061216652
Loss: 0.0027413242496550083
Loss: 0.0009390637278556824
Loss: 0.00283817108720541
Loss: 0.0025788741186261177
Loss: 0.0007827990921214223
Loss: 0.00020363630028441548
Loss: 0.00021183807984925807
Loss: 0.0005019143573008478
Loss: 0.00023706944193691015
Loss: 0.00015938599244691432
Loss: 0.0001047758269123733
Loss: 9.248482092516497e-05


In [ ]:
model.load_state_dict(torch.load(project_path + "roberta/model_embed_2.pt"))

<All keys matched successfully>

In [ ]:
# def decode(pred):
#   decoded = torch.where(pred >= 0.5, 1, 0)
#   return decoded

test_sample = data_test


results = []
labels_ = []
total_acc = 0
test_batch_size = 1
for i in range(0, len(test_sample), test_batch_size):

  input_ids, input_mask, labels = tokenize_batch_farn(test_sample[i : i+test_batch_size])
  labels = torch.tensor(labels).unsqueeze(-1).to(dtype=torch.float32).cuda()

  preds = model.forward(input_ids, input_mask)
  decoded = decode(preds).to(dtype=torch.float32)

  results.append(decoded)
  labels_.append(labels)
  acc = torch.where(decoded == labels, 1, 0)
  total_acc += acc.sum()

print(int(total_acc) / len(test_sample))


0.9993319229484467


In [ ]:

results = torch.cat(results).cpu()

labels = torch.cat(labels_).cpu()

print(results.shape, labels.shape)
results = results.reshape(-1)
labels = labels.reshape(-1)
print(results.shape, labels.shape)

from sklearn.metrics import precision_score, recall_score, f1_score

# accuracy = (results == labels).sum() / len(labels)

prec = precision_score(labels, results)
recall = recall_score(labels, results)
f1 = f1_score(labels, results)

print(acc, prec, recall, f1)

# true_pos = torch.logical_and([results, labels])
# false_pos = torch.logical_and([results, torch.logical_not(labels)])
# false_neg = torch.logical_and([labels, torch.logical_not(results)])

# precision = true_pos.sum() / (true_pos.sum() + false_pos.sum())
# recall = true_pos.sum() / (true_pos.sum() + false_neg.sum())





torch.Size([8981, 1]) torch.Size([8981, 1])
torch.Size([8981]) torch.Size([8981])
tensor([[1]], device='cuda:0') 1.0 0.9985524728588661 0.9992757122163206
